In [1]:
# prompt: 從google drive匯入zip檔並解壓縮

from google.colab import drive
import zipfile

drive.mount('/content/drive')

# Replace 'your_zip_file.zip' with the actual name of your zip file
# Replace '/content/drive/MyDrive/path/to/your/zip/file' with the actual path to your zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/SLA/SLAoptuna_train.zip'

# Replace '/content/extracted_files' with the desired extraction directory
extract_to_path = '/content'

try:
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)
    print(f"Successfully extracted files to {extract_to_path}")

except FileNotFoundError:
    print(f"Error: Zip file not found at {zip_file_path}. Please check the file path.")
except zipfile.BadZipFile:
    print(f"Error: Invalid zip file. {zip_file_path} may be corrupted.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Mounted at /content/drive
Successfully extracted files to /content


In [2]:
# Replace 'your_zip_file.zip' with the actual name of your zip file
# Replace '/content/drive/MyDrive/path/to/your/zip/file' with the actual path to your zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/SLA/SLA.zip'

# Replace '/content/extracted_files' with the desired extraction directory
extract_to_path = '/content'

try:
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)
    print(f"Successfully extracted files to {extract_to_path}")

except FileNotFoundError:
    print(f"Error: Zip file not found at {zip_file_path}. Please check the file path.")
except zipfile.BadZipFile:
    print(f"Error: Invalid zip file. {zip_file_path} may be corrupted.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Successfully extracted files to /content


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from sla_dataset_GPU import SLADataset  # Time: 13.651094913482666
# from sla_dataset import SLADataset  # Time: 866.5504276752472
from sklearn.metrics import f1_score
import os
import json
from datetime import datetime
import numpy as np
import random
import time

# 定義模型架構
class DynamicCNN(nn.Module):
    def __init__(self, conv1_out, conv2_out, conv3_out, dropout_rate, num_classes):
        super(DynamicCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, conv1_out, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(conv1_out, conv2_out, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(conv2_out, conv3_out, kernel_size=3, padding=1)
        self.fc = nn.Linear(conv3_out * 12 * 20, num_classes)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 設置亂數種子
def set_seed(seed=42):
    random.seed(seed)  # Python 內建隨機數生成器
    np.random.seed(seed)  # NumPy 隨機數生成器
    torch.manual_seed(seed)  # PyTorch CPU 隨機數生成器
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)  # PyTorch GPU 隨機數生成器
        torch.cuda.manual_seed_all(seed)  # 多 GPU 的隨機數生成器

# 定义超参数
num_epochs = 100
fold_count = 5      # 总共有 5 折
learning_rate = 0.000314430987067442 # 0.000314430987067442
batch_size = 64
dropout_rate = 0.11218913608873872
conv1_out_channels = 48
conv2_out_channels = 96
conv3_out_channels = 128
weight_decay = 9.344674979883828e-05

# 定义测站地名
locations = [
    "Atlantic City",
    "Baltimore",
    "Eastport",
    "Fort Pulaski",
    "Lewes",
    "New London",
    "Newport",
    "Portland",
    "Sandy Hook",
    "Sewells Point",
    "The Battery",
    "Washington",
]
set_seed(42)
# 数据目录
nc_data_dir = "/content/SLA/Copernicus_ENA_Satelite Maps_Training_Data"
label_data_dir = "/content/SLA/Training_Anomalies_Station Data"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"model_results_{timestamp}"

# 创建结果保存目录
os.makedirs(output_dir, exist_ok=True)

# 初始化数据集
sla_dataset = SLADataset(nc_data_dir, label_data_dir, locations, fill_value=0.0)
print(sla_dataset.all_dates[0:10])

# 切分训练集和测试集
dataset_size = len(sla_dataset)
fold_size = dataset_size // fold_count  # 每折的测试数据大小
test_size = fold_size

# 随机打乱数据索引
indices = list(range(dataset_size))
# random.shuffle(indices)  # 随机打乱索引

# 按五折划分索引
folds = [indices[i * fold_size:(i + 1) * fold_size] for i in range(fold_count)]

for k in range(1, fold_count + 1):

  test_indices = folds[k - 1]

  train_indices = [idx for fold in folds if fold != test_indices for idx in fold]

  # 划分数据集
  train_dataset = Subset(sla_dataset, train_indices)
  test_dataset = Subset(sla_dataset, test_indices)

  set_seed(42+k)
  # 创建 DataLoader
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  # 检查是否可以使用 GPU
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # 初始化模型、损失函数和优化器
  model = DynamicCNN(conv1_out_channels, conv2_out_channels, conv3_out_channels, dropout_rate, num_classes=len(locations)).to(device)

  # 预先计算正样本权重
  train_size = len(train_indices) # len(train_dataset)
  label_sum = torch.zeros((12,),dtype=torch.float64).cuda()
  for i, (sla, labels) in enumerate(train_loader):
      label_sum+=labels.sum(axis=0)
  positive_weight = (train_size-label_sum)/label_sum
  positive_weight[6] = positive_weight[6] * 0.0
  # positive_weight = torch.where(positive_weight > 100, torch.tensor(100), positive_weight)
  print(positive_weight)

  criterion = nn.BCEWithLogitsLoss(pos_weight=positive_weight)  # 改为二分类的损失函数
  optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

  def evaluate_model(model, loader):
      model.eval()
      total_loss = 0.0
      all_labels = []
      all_preds = []
      with torch.no_grad():
          for sla, labels in loader:
              if sla.device != torch.device(device) or sla.dtype != torch.float32:
                  sla = sla.to(device, dtype=torch.float32)
              if labels.device != torch.device(device) or labels.dtype != torch.float32:
                  labels = labels.to(device, dtype=torch.float32)

              outputs = model(sla)
              loss = criterion(outputs, labels)
              total_loss += loss.item()

              preds = torch.sigmoid(outputs).cpu().numpy() > 0.5
              all_labels.append(labels.cpu().numpy())
              all_preds.append(preds)

      all_labels = np.concatenate(all_labels, axis=0)
      all_preds = np.concatenate(all_preds, axis=0)

      # 计算评价指标
      tpr = np.mean([
          np.sum((all_labels[:, i] == 1) & (all_preds[:, i] == 1)) / np.sum(all_labels[:, i] == 1)
          if np.sum(all_labels[:, i] == 1) > 0 else 0
          for i in range(all_labels.shape[1])
      ])
      fpr = np.mean([
          np.sum((all_labels[:, i] == 0) & (all_preds[:, i] == 1)) / np.sum(all_labels[:, i] == 0)
          if np.sum(all_labels[:, i] == 0) > 0 else 0
          for i in range(all_labels.shape[1])
      ])
      f1 = f1_score(all_labels, all_preds, average="macro")

      avg_loss = total_loss / len(loader)
      return avg_loss, tpr, fpr, f1

  # 训练循环
  start_time = time.time()
  results = []
  # 保存F1-Score最高的模型
  best_f1_score = 0.0
  best_model_path = os.path.join(output_dir, str(k)+"best_sla_cnn_model.pth")
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0

      for i, (sla, labels) in enumerate(train_loader):
          # sla = sla.to(device, dtype=torch.float32)  # 确保输入形状符合要求
          # labels = labels.to(device, dtype=torch.float32)  # 将标签转换为 float 类型以匹配 BCEWithLogitsLoss
          if sla.device != torch.device(device) or sla.dtype != torch.float32:
                  sla = sla.to(device, dtype=torch.float32)
          if labels.device != torch.device(device) or labels.dtype != torch.float32:
                  labels = labels.to(device, dtype=torch.float32)

          # 前向传播
          outputs = model(sla)
          loss = criterion(outputs, labels)

          # 反向传播和优化
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

          if (i + 1) % 10 == 0:
              print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

      avg_train_loss = running_loss / len(train_loader)
      print(f"Folder {k} Epoch [{epoch + 1}/{num_epochs}], Average Training Loss: {avg_train_loss:.4f}")

      # 测试并评估模型
      if test_size>0:
          test_loss, test_tpr, test_fpr, test_f1 = evaluate_model(model, test_loader)
          print(f"Folder {k} Epoch [{epoch + 1}/{num_epochs}], Test Loss: {test_loss:.4f}, Test TPR: {test_tpr:.4f}, Test FPR: {test_fpr:.4f}, Test F1-Score: {test_f1:.4f}")

          # 检查是否为当前最高 F1-Score
          if test_f1 > best_f1_score:
              best_f1_score = test_f1
              torch.save(model.state_dict(), best_model_path)
              print(f"Folder {k} New best model saved with F1-Score: {best_f1_score:.4f}")

          # 保存结果
          results.append({
              "epoch": epoch + 1,
              "test_loss": test_loss,
              "test_tpr": test_tpr,
              "test_fpr": test_fpr,
              "test_f1": test_f1,
          })
  print("Time:",time.time()-start_time)
  # 保存模型和结果
  model_path = os.path.join(output_dir, str(k)+"sla_cnn_model.pth")
  torch.save(model.state_dict(), model_path)
  print(f"Model saved as {model_path}")

  results_path = os.path.join(output_dir, str(k)+"results.json")
  with open(results_path, "w") as f:
      json.dump(results, f, indent=4)
  print(f"Results saved as {results_path}")

  # 保存模型架构
  architecture_path = os.path.join(output_dir, str(k)+"model_architecture.txt")
  with open(architecture_path, "w") as f:
      f.write(str(model))
  print(f"Model architecture saved as {architecture_path}")


['19930101', '19930102', '19930103', '19930104', '19930105', '19930106', '19930107', '19930108', '19930109', '19930110']
tensor([ 21.5483,  36.4359,  15.7816,  64.6180,  21.2901, 215.2963,   0.0000,
         76.8667,  19.7829,  57.4000, 128.7778,  15.6382], device='cuda:0',
       dtype=torch.float64)
Epoch [1/100], Step [10/92], Loss: 1.3884
Epoch [1/100], Step [20/92], Loss: 1.0842
Epoch [1/100], Step [30/92], Loss: 1.2051
Epoch [1/100], Step [40/92], Loss: 1.3096
Epoch [1/100], Step [50/92], Loss: 1.0883
Epoch [1/100], Step [60/92], Loss: 1.0937
Epoch [1/100], Step [70/92], Loss: 1.5737
Epoch [1/100], Step [80/92], Loss: 1.1732
Epoch [1/100], Step [90/92], Loss: 1.1030
Folder 1 Epoch [1/100], Average Training Loss: 1.2709
Folder 1 Epoch [1/100], Test Loss: 1.0549, Test TPR: 0.8333, Test FPR: 0.8333, Test F1-Score: 0.0386
Folder 1 New best model saved with F1-Score: 0.0386
Epoch [2/100], Step [10/92], Loss: 1.0110
Epoch [2/100], Step [20/92], Loss: 1.2697
Epoch [2/100], Step [30/92],

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Folder 2 Epoch [1/100], Test Loss: 1.0395, Test TPR: 0.5793, Test FPR: 0.5453, Test F1-Score: 0.0299
Folder 2 New best model saved with F1-Score: 0.0299
Epoch [2/100], Step [10/92], Loss: 1.4707
Epoch [2/100], Step [20/92], Loss: 1.5177
Epoch [2/100], Step [30/92], Loss: 1.4000
Epoch [2/100], Step [40/92], Loss: 1.0329
Epoch [2/100], Step [50/92], Loss: 0.8530
Epoch [2/100], Step [60/92], Loss: 1.2774
Epoch [2/100], Step [70/92], Loss: 1.1947
Epoch [2/100], Step [80/92], Loss: 1.0454
Epoch [2/100], Step [90/92], Loss: 1.0066
Folder 2 Epoch [2/100], Average Training Loss: 1.1998


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Folder 2 Epoch [2/100], Test Loss: 1.0285, Test TPR: 0.3963, Test FPR: 0.3196, Test F1-Score: 0.0568
Folder 2 New best model saved with F1-Score: 0.0568
Epoch [3/100], Step [10/92], Loss: 1.0189
Epoch [3/100], Step [20/92], Loss: 1.4517
Epoch [3/100], Step [30/92], Loss: 0.8798
Epoch [3/100], Step [40/92], Loss: 0.6477
Epoch [3/100], Step [50/92], Loss: 1.7540
Epoch [3/100], Step [60/92], Loss: 0.7734
Epoch [3/100], Step [70/92], Loss: 0.7533
Epoch [3/100], Step [80/92], Loss: 1.2700
Epoch [3/100], Step [90/92], Loss: 1.1752
Folder 2 Epoch [3/100], Average Training Loss: 1.1006


...

Folder 3 Epoch [98/100], Test Loss: 4.8731, Test TPR: 0.0648, Test FPR: 0.0184, Test F1-Score: 0.0524
Epoch [99/100], Step [10/92], Loss: 0.2165
Epoch [99/100], Step [20/92], Loss: 0.1746
Epoch [99/100], Step [30/92], Loss: 0.1943
Epoch [99/100], Step [40/92], Loss: 0.1968
Epoch [99/100], Step [50/92], Loss: 0.2126
Epoch [99/100], Step [60/92], Loss: 0.2392
Epoch [99/100], Step [70/92], Loss: 0.1836
Epoch [99/100], Step [80/92], Loss: 0.2007
Epoch [99/100], Step [90/92], Loss: 0.1699
Folder 3 Epoch [99/100], Average Training Loss: 0.2112


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Folder 3 Epoch [99/100], Test Loss: 4.7414, Test TPR: 0.0761, Test FPR: 0.0200, Test F1-Score: 0.0634
Epoch [100/100], Step [10/92], Loss: 0.2590
Epoch [100/100], Step [20/92], Loss: 0.3866
Epoch [100/100], Step [30/92], Loss: 0.2304
Epoch [100/100], Step [40/92], Loss: 0.2052
Epoch [100/100], Step [50/92], Loss: 0.2732
Epoch [100/100], Step [60/92], Loss: 0.2060
Epoch [100/100], Step [70/92], Loss: 0.2225
Epoch [100/100], Step [80/92], Loss: 0.1463
Epoch [100/100], Step [90/92], Loss: 0.1438
Folder 3 Epoch [100/100], Average Training Loss: 0.2036


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Folder 3 Epoch [100/100], Test Loss: 4.8306, Test TPR: 0.0766, Test FPR: 0.0181, Test F1-Score: 0.0598
Time: 484.6536328792572
Model saved as model_results_20250131_015052/3sla_cnn_model.pth
Results saved as model_results_20250131_015052/3results.json
Model architecture saved as model_results_20250131_015052/3model_architecture.txt
tensor([ 24.0644,  43.2424,  14.1688,  71.0988,  25.0714, 252.9130,   0.0000,
         83.6377,  21.8125,  71.0988, 165.8571,  16.3810], device='cuda:0',
       dtype=torch.float64)
Epoch [1/100], Step [10/92], Loss: 0.8679
Epoch [1/100], Step [20/92], Loss: 1.2044
Epoch [1/100], Step [30/92], Loss: 1.9492
Epoch [1/100], Step [40/92], Loss: 1.2771
Epoch [1/100], Step [50/92], Loss: 1.8997
Epoch [1/100], Step [60/92], Loss: 1.1931
Epoch [1/100], Step [70/92], Loss: 1.0978
Epoch [1/100], Step [80/92], Loss: 0.9195
Epoch [1/100], Step [90/92], Loss: 1.4595
Folder 4 Epoch [1/100], Average Training Loss: 1.2583
Folder 4 Epoch [1/100], Test Loss: 1.3778, Test TPR:

In [4]:
# # prompt: 複製model_path到/content/sla_cnn_model.pth

# import shutil
# import os

# # Define the source and destination paths
# # model_path = "/content/model_results_20240308_144757/sla_cnn_model.pth"  # Replace with the actual path to your model file
# destination_path = "/content/sla_cnn_model.pth"

# # Check if the source file exists
# if os.path.exists(model_path):
#     try:
#         # Copy the file
#         shutil.copyfile(model_path, destination_path)
#         print(f"Successfully copied '{model_path}' to '{destination_path}'")
#     except Exception as e:
#         print(f"An error occurred during file copying: {e}")
# else:
#     print(f"Error: The source file '{model_path}' does not exist.")

In [5]:
# prompt: 下載資料夾output_dir

from google.colab import files
import shutil

# Define the source and destination paths
source_dir = output_dir  # Replace with the actual path to your directory
destination_zip = "output_dir.zip"

# Create a zip archive of the directory
shutil.make_archive("output_dir", 'zip', source_dir)

# prompt: 移動檔案destination_zip到'/content/drive/MyDrive/SLA/output_dir.zip'

# Define the source and destination paths
destination_path = "/content/drive/MyDrive/SLA/output_dir.zip"

# Copy the file
shutil.copyfile(destination_zip, destination_path)

# Download the zip file
# files.download(destination_zip)

'/content/drive/MyDrive/SLA/output_dir.zip'

In [8]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import pandas as pd

# class Model:
#     def __init__(self):
#         print("Dummy Model initialized")
#         # Put your initialization code here
#         # load the save model here


#     def predict(self, X):
#         # 要處裡nan
#         # Put your prediction code here
#         # This example predicts a random value for 12 station
#         # The output should be a dataframe with 10 rows and 12 columns
#         # Each value should be 1 for anamoly and 0 for normal
#         # Return a np array of 1s and 0s with the same length of 12
#         # with random prediction of 1 or 0
#         return torch.randint(0, 2, (12,)).numpy()
import torch
import torch.nn as nn
import numpy as np
from scipy.stats import mode  # 统计众数

class Model:
    def __init__(self):
        print("Loading trained model...")
        # Load the saved model
        self.num_classes = 12
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        locations = [
                "Atlantic City",
                "Baltimore",
                "Eastport",
                "Fort Pulaski",
                "Lewes",
                "New London",
                "Newport",
                "Portland",
                "Sandy Hook",
                "Sewells Point",
                "The Battery",
                "Washington",
            ]
        dropout_rate = 0.11218913608873872
        conv1_out_channels = 48
        conv2_out_channels = 96
        conv3_out_channels = 128
        self.folder_count = 5
        self.models = []
        for fold in range(1, fold_count + 1):
          model = DynamicCNN(conv1_out_channels, conv2_out_channels, conv3_out_channels, dropout_rate, num_classes=len(locations)).to(self.device)
          model.load_state_dict(torch.load(os.path.join(output_dir, str(fold)+"best_sla_cnn_model.pth"), map_location=self.device))
          model.eval()
          self.models.append(model)

    def preprocess(self, X):
        # Replace NaN values with 0
        X = np.nan_to_num(X, nan=0.0)
        # Ensure the shape is compatible with the model
        X = torch.tensor(X, dtype=torch.float32).unsqueeze(0).to(self.device)  # Add batch and channel dims
        return X

    def predict(self, X):
        # Preprocess the input
        X_processed = self.preprocess(X)
        # Make predictions
        preds = []
        for fold in range(1, fold_count + 1):
          with torch.no_grad():
              outputs = self.models[fold-1](X_processed)
              pred = torch.sigmoid(outputs).cpu().numpy() > 0.5  # Binary prediction
              pred = pred.reshape(1,12)
              preds.append(pred)
        preds = np.concatenate(preds, axis=0).astype(np.int64)
        return mode(preds, axis=0).mode.flatten()


In [9]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sla_dataset import SLADataset
# from model import Model

# 初始化数据目录和模型
nc_data_dir = "/content/SLA/Copernicus_ENA_Satelite Maps_Training_Data"
label_data_dir = "/content/SLA/Training_Anomalies_Station Data"
output_csv_path = "my_sla.csv"

# 测站地名
locations = [
    "Atlantic City",
    "Baltimore",
    "Eastport",
    "Fort Pulaski",
    "Lewes",
    "New London",
    "Newport",
    "Portland",
    "Sandy Hook",
    "Sewells Point",
    "The Battery",
    "Washington",
]

# 初始化数据集
sla_dataset = SLADataset(nc_data_dir, label_data_dir, locations)
model = Model()

# 收集预测结果和真实标签
results = []
true_labels = []

for idx in range(len(sla_dataset)):
    sla, labels = sla_dataset[idx]
    predictions = model.predict(sla)

    # Format date
    raw_date = sla_dataset.all_dates[idx]
    formatted_date = f"{raw_date[4:6]}-{raw_date[6:]}-{raw_date[:4]}"  # Format YYYY-MM-DD

    results.append([formatted_date] + predictions.tolist())
    true_labels.append(labels.tolist())

# 转换为 DataFrame
columns = ["Date"] + locations
predictions_df = pd.DataFrame(results, columns=columns)

true_labels_df = pd.DataFrame(true_labels, columns=locations)
true_labels_df.insert(0, "Date", [f"{d[4:6]}-{d[6:]}-{d[:4]}" for d in sla_dataset.all_dates])

# 保存预测结果
predictions_df.to_csv(output_csv_path, index=False)
print(f"Predictions saved to {output_csv_path}")

# 计算 TPR, FPR, F1-Score
tpr_list, fpr_list, f1_list = [], [], []

for station in locations:
    y_true = true_labels_df[station]
    y_pred = predictions_df[station]

    tp = ((y_true == 1) & (y_pred == 1)).sum()
    fp = ((y_true == 0) & (y_pred == 1)).sum()
    tn = ((y_true == 0) & (y_pred == 0)).sum()
    fn = ((y_true == 1) & (y_pred == 0)).sum()

    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    f1 = f1_score(y_true, y_pred)

    tpr_list.append(tpr)
    fpr_list.append(fpr)
    f1_list.append(f1)

# 打印平均指标
avg_tpr = np.mean(tpr_list)
avg_fpr = np.mean(fpr_list)
avg_f1 = np.mean(f1_list)

print(f"Average TPR: {avg_tpr:.4f}")
print(f"Average FPR: {avg_fpr:.4f}")
print(f"Average F1-Score: {avg_f1:.4f}")


Loading trained model...


<ipython-input-8-a51dba993fe5>:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(output_dir, str(fold)+"best_sla_cnn_model.pth"

Predictions saved to my_sla.csv
Average TPR: 0.8718
Average FPR: 0.0790
Average F1-Score: 0.3250


In [10]:
# prompt: 下載output_csv_path

from google.colab import files
# files.download('my_sla.csv')

In [11]:
shutil.copyfile('my_sla.csv', "/content/drive/MyDrive/SLA/my_sla.csv")
print(f"Successfully copied '{destination_zip}' to '{destination_path}'")

Successfully copied 'output_dir.zip' to '/content/drive/MyDrive/SLA/output_dir.zip'
